In [28]:
import pandas as pd
from fbprophet import Prophet
from  datetime import datetime, timedelta
import gc
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [29]:
cal_dtype = {"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
              "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
              "month": "int16", "year": "int16", "snap_CA": "int8", 'snap_TX': 'int8', 'snap_WI': 'int8'}
price_dtype = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32"}

In [30]:
dt = pd.read_csv('sales_train_validation.csv')
dt = dt.astype({col: 'int32' for col in dt.select_dtypes('int64').columns})

In [31]:
cal = pd.read_csv('calendar.csv', dtype = cal_dtype)
cal['date'] = pd.to_datetime(cal['date'])
cal = cal.astype({col: 'int32' for col in cal.select_dtypes('int64').columns})
cal_1 = cal[0:1913]
future = cal[1913:1941]

In [32]:
hol_1 = cal[['event_name_1', 'date']]
hol_1 = hol_1.dropna()
hol_1 = hol_1.rename(columns={'date': 'ds', 'event_name_1': 'holiday'})
hol_2 = cal[['event_name_2', 'date']]
hol_2 = hol_2.dropna()
hol_2 = hol_2.rename(columns={'date': 'ds', 'event_name_2': 'holiday'})
hol = pd.concat([hol_1, hol_2], axis=0, sort=False)
hol['lower_window'] = 0
hol['upper_window'] = 1

In [33]:
dt_st_dep = dt.groupby(['store_id', 'dept_id'], as_index=False).sum()

In [34]:
dt_y_pred = dt[['id','dept_id','store_id', 'd_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891',
               'd_1892', 'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898',
               'd_1899', 'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
               'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913',]]
d_id = dt_y_pred.dept_id.unique()
d_id_lst = d_id.tolist()
st_id = dt_y_pred.store_id.unique()
st_id_lst = st_id.tolist()

In [35]:
df_sub = pd.DataFrame(columns = ['id','dept_id','store_id', 'd_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891',
               'd_1892', 'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898',
               'd_1899', 'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
               'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913',])

In [36]:
for st_id in st_id_lst:
    for dp_id in d_id_lst:
        d_id = dp_id
        s_id = st_id
        print(s_id, d_id)
        hob1_st1 = dt_y_pred.loc[(dt_y_pred['dept_id'] == d_id) & (dt_y_pred['store_id'] == s_id)]

        dt_st_dep_n = dt_st_dep.loc[(dt_st_dep['dept_id'] == d_id) & (dt_st_dep['store_id'] == s_id)]
        div_coeff_28 = dt_st_dep_n[['store_id','dept_id', 'd_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891',
                       'd_1892', 'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898',
                       'd_1899', 'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
                       'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913',]]
        div_coeff_28 = div_coeff_28.reset_index(drop=True)
        div_coeff_lst = div_coeff_28.loc[0, :].values.tolist()

        hob1_st1['d_1886'] = hob1_st1['d_1886']/div_coeff_lst[2]
        hob1_st1['d_1887'] = hob1_st1['d_1887']/div_coeff_lst[3]
        hob1_st1['d_1888'] = hob1_st1['d_1888']/div_coeff_lst[4]
        hob1_st1['d_1889'] = hob1_st1['d_1889']/div_coeff_lst[5]
        hob1_st1['d_1890'] = hob1_st1['d_1890']/div_coeff_lst[6]
        hob1_st1['d_1891'] = hob1_st1['d_1891']/div_coeff_lst[7]
        hob1_st1['d_1892'] = hob1_st1['d_1892']/div_coeff_lst[8]
        hob1_st1['d_1893'] = hob1_st1['d_1893']/div_coeff_lst[9]
        hob1_st1['d_1894'] = hob1_st1['d_1894']/div_coeff_lst[10]
        hob1_st1['d_1895'] = hob1_st1['d_1895']/div_coeff_lst[11]
        hob1_st1['d_1896'] = hob1_st1['d_1896']/div_coeff_lst[12]
        hob1_st1['d_1897'] = hob1_st1['d_1897']/div_coeff_lst[13]
        hob1_st1['d_1898'] = hob1_st1['d_1898']/div_coeff_lst[14]
        hob1_st1['d_1899'] = hob1_st1['d_1899']/div_coeff_lst[15]
        hob1_st1['d_1900'] = hob1_st1['d_1900']/div_coeff_lst[16]
        hob1_st1['d_1901'] = hob1_st1['d_1901']/div_coeff_lst[17]
        hob1_st1['d_1902'] = hob1_st1['d_1902']/div_coeff_lst[18]
        hob1_st1['d_1903'] = hob1_st1['d_1903']/div_coeff_lst[19]
        hob1_st1['d_1904'] = hob1_st1['d_1904']/div_coeff_lst[20]
        hob1_st1['d_1905'] = hob1_st1['d_1905']/div_coeff_lst[21]
        hob1_st1['d_1906'] = hob1_st1['d_1906']/div_coeff_lst[22]
        hob1_st1['d_1907'] = hob1_st1['d_1907']/div_coeff_lst[23]
        hob1_st1['d_1908'] = hob1_st1['d_1908']/div_coeff_lst[24]
        hob1_st1['d_1909'] = hob1_st1['d_1909']/div_coeff_lst[25]
        hob1_st1['d_1910'] = hob1_st1['d_1910']/div_coeff_lst[26]
        hob1_st1['d_1911'] = hob1_st1['d_1911']/div_coeff_lst[27]
        hob1_st1['d_1912'] = hob1_st1['d_1912']/div_coeff_lst[28]
        hob1_st1['d_1913'] = hob1_st1['d_1913']/div_coeff_lst[29]


        catcols = ['store_id', 'dept_id']
        dt_st_dep_n = pd.melt(dt_st_dep_n, 
                            id_vars = catcols, 
                            value_vars = [col for col in dt_st_dep_n.columns if col.startswith('d_')], 
                            var_name = 'd', 
                            value_name = 'sales')
        dt_st_dep_n = dt_st_dep_n.merge(cal, on= "d", copy = False)
        df_train = dt_st_dep_n

        tr_chk = df_train[['date', 'sales']]
        tr_chk = tr_chk.rename(columns={'date': 'ds', 'sales': 'y'})
        fut = future[['date']]
        fut = fut.rename(columns={'date': 'ds'})
        m = Prophet(holidays=hol, daily_seasonality=True)
        if (s_id.split('_')[0] == 'CA'):
            tr_chk['snap'] = df_train['snap_CA']
            fut['snap'] = future['snap_CA']
        elif (s_id.split('_')[0] == 'TX'):
            tr_chk['snap'] = df_train['snap_TX']
            fut['snap'] = future['snap_TX']
        else:
            tr_chk['snap'] = df_train['snap_WI']
            fut['snap'] = future['snap_WI']
        m.add_regressor('snap')
        forecast = m.fit(tr_chk).predict(fut)
        y_pred = forecast['yhat'].values.tolist()

        hob1_st1['d_1886'] = hob1_st1['d_1886']*y_pred[0]
        hob1_st1['d_1887'] = hob1_st1['d_1887']*y_pred[1]
        hob1_st1['d_1888'] = hob1_st1['d_1888']*y_pred[2]
        hob1_st1['d_1889'] = hob1_st1['d_1889']*y_pred[3]
        hob1_st1['d_1890'] = hob1_st1['d_1890']*y_pred[4]
        hob1_st1['d_1891'] = hob1_st1['d_1891']*y_pred[5]
        hob1_st1['d_1892'] = hob1_st1['d_1892']*y_pred[6]
        hob1_st1['d_1893'] = hob1_st1['d_1893']*y_pred[7]
        hob1_st1['d_1894'] = hob1_st1['d_1894']*y_pred[8]
        hob1_st1['d_1895'] = hob1_st1['d_1895']*y_pred[9]
        hob1_st1['d_1896'] = hob1_st1['d_1896']*y_pred[10]
        hob1_st1['d_1897'] = hob1_st1['d_1897']*y_pred[11]
        hob1_st1['d_1898'] = hob1_st1['d_1898']*y_pred[12]
        hob1_st1['d_1899'] = hob1_st1['d_1899']*y_pred[13]
        hob1_st1['d_1900'] = hob1_st1['d_1900']*y_pred[14]
        hob1_st1['d_1901'] = hob1_st1['d_1901']*y_pred[15]
        hob1_st1['d_1902'] = hob1_st1['d_1902']*y_pred[16]
        hob1_st1['d_1903'] = hob1_st1['d_1903']*y_pred[17]
        hob1_st1['d_1904'] = hob1_st1['d_1904']*y_pred[18]
        hob1_st1['d_1905'] = hob1_st1['d_1905']*y_pred[19]
        hob1_st1['d_1906'] = hob1_st1['d_1906']*y_pred[20]
        hob1_st1['d_1907'] = hob1_st1['d_1907']*y_pred[21]
        hob1_st1['d_1908'] = hob1_st1['d_1908']*y_pred[22]
        hob1_st1['d_1909'] = hob1_st1['d_1909']*y_pred[23]
        hob1_st1['d_1910'] = hob1_st1['d_1910']*y_pred[24]
        hob1_st1['d_1911'] = hob1_st1['d_1911']*y_pred[25]
        hob1_st1['d_1912'] = hob1_st1['d_1912']*y_pred[26]
        hob1_st1['d_1913'] = hob1_st1['d_1913']*y_pred[27]

        df_sub = pd.concat([df_sub, hob1_st1], axis=0, sort=False)

CA_1 HOBBIES_1
CA_1 HOBBIES_2
CA_1 HOUSEHOLD_1
CA_1 HOUSEHOLD_2
CA_1 FOODS_1
CA_1 FOODS_2
CA_1 FOODS_3
CA_2 HOBBIES_1
CA_2 HOBBIES_2
CA_2 HOUSEHOLD_1
CA_2 HOUSEHOLD_2
CA_2 FOODS_1
CA_2 FOODS_2
CA_2 FOODS_3
CA_3 HOBBIES_1
CA_3 HOBBIES_2
CA_3 HOUSEHOLD_1
CA_3 HOUSEHOLD_2
CA_3 FOODS_1
CA_3 FOODS_2
CA_3 FOODS_3
CA_4 HOBBIES_1
CA_4 HOBBIES_2
CA_4 HOUSEHOLD_1
CA_4 HOUSEHOLD_2
CA_4 FOODS_1
CA_4 FOODS_2
CA_4 FOODS_3
TX_1 HOBBIES_1
TX_1 HOBBIES_2
TX_1 HOUSEHOLD_1
TX_1 HOUSEHOLD_2
TX_1 FOODS_1
TX_1 FOODS_2
TX_1 FOODS_3
TX_2 HOBBIES_1
TX_2 HOBBIES_2
TX_2 HOUSEHOLD_1
TX_2 HOUSEHOLD_2
TX_2 FOODS_1
TX_2 FOODS_2
TX_2 FOODS_3
TX_3 HOBBIES_1
TX_3 HOBBIES_2
TX_3 HOUSEHOLD_1
TX_3 HOUSEHOLD_2
TX_3 FOODS_1
TX_3 FOODS_2
TX_3 FOODS_3
WI_1 HOBBIES_1
WI_1 HOBBIES_2
WI_1 HOUSEHOLD_1
WI_1 HOUSEHOLD_2
WI_1 FOODS_1
WI_1 FOODS_2
WI_1 FOODS_3
WI_2 HOBBIES_1
WI_2 HOBBIES_2
WI_2 HOUSEHOLD_1
WI_2 HOUSEHOLD_2
WI_2 FOODS_1
WI_2 FOODS_2
WI_2 FOODS_3
WI_3 HOBBIES_1
WI_3 HOBBIES_2
WI_3 HOUSEHOLD_1
WI_3 HOUSEHOLD_2
WI_3 FOOD

In [37]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30490 entries, 0 to 30489
Data columns (total 31 columns):
id          30490 non-null object
dept_id     30490 non-null object
store_id    30490 non-null object
d_1886      30490 non-null float64
d_1887      30490 non-null float64
d_1888      30490 non-null float64
d_1889      30490 non-null float64
d_1890      30490 non-null float64
d_1891      30490 non-null float64
d_1892      30490 non-null float64
d_1893      30490 non-null float64
d_1894      30490 non-null float64
d_1895      30490 non-null float64
d_1896      30490 non-null float64
d_1897      30490 non-null float64
d_1898      30490 non-null float64
d_1899      30490 non-null float64
d_1900      30490 non-null float64
d_1901      30490 non-null float64
d_1902      30490 non-null float64
d_1903      30490 non-null float64
d_1904      30490 non-null float64
d_1905      30490 non-null float64
d_1906      30490 non-null float64
d_1907      30490 non-null float64
d_1908      30490 n

In [38]:
df_sub_2 = df_sub.drop(['store_id', 'dept_id'], axis=1)  
COLUMN_NAMES = ['id', 'F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17','F18','F19',
               'F20', 'F21','F22','F23','F24','F25','F26','F27','F28']
df_sub_2.columns = COLUMN_NAMES
df_sub_c = df_sub_2.copy()
df_sub_c["id"] = df_sub_c["id"].str.replace("validation$", "evaluation")
df_sub_2 = pd.concat([df_sub_2, df_sub_c], axis=0, sort=False)
df_sub_2.to_csv("submission_pro_snap.csv",index=False)

In [39]:
df_sub_2.id.nunique(), df_sub_2["id"].str.contains("validation$").sum()

(60980, 30490)

In [40]:
df_sub_2.shape

(60980, 29)